In [ ]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install pymysql

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 크롤링한 데이터중 머신러닝 엔지니어 관련 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/pro/개인/pro_data/wanted_ML_engineer.csv')

df = df.fillna('')

# 스킬 리스트 전처리
skill_list = []
for i in range(len(df['스킬'])):
    skill_list = skill_list + df.loc[i,'스킬'].split(',')

count = Counter(skill_list)
stopwords = [''] ##### 개별적 불용어 추가
# 기술컬럼 데이터를 본 결과 불용어를 딱히 추가할 필요성을 느끼지 못해 사용하지 않았음
# 혹시 불용어 추가를 하려면 추가하면됨
# 불용어 처리
remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1})
remove_char_counter = Counter({x:remove_char_counter[x] for x in remove_char_counter if x not in stopwords})

remove_char_counter.most_common(50)

df = pd.DataFrame(remove_char_counter.most_common(50))
df
# 상위 50개만 뽑아서 데이터 프레임으로 변환

wc = WordCloud(font_path='/content/drive/MyDrive/malgun.ttf', width=1200, height=500,
               scale=4.0, max_font_size=600, background_color='white')
gen = wc.generate_from_frequencies(remove_char_counter)
# 워드 클라우드 설정

# 워드클라우드 그리기 및 파일로 저장
plt.figure(figsize=(15, 7))
plt.imshow(gen)
plt.axis('off')
plt.savefig(f'/content/drive/MyDrive/AICC_WEEK04/.png')

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pymysql
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#사이트 메인화면에서 스크롤 내리기 
def page_down():
    global error
    global end_crawling
    while True:
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)
        current_scroll = driver.execute_script('return window.pageYOffset')
        driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
        time.sleep(1)
        driver.execute_script(f'window.scrollTo(0, {current_scroll + 150});')
        time.sleep(1)
        down_scroll = driver.execute_script('return window.pageYOffset')
        time.sleep(1)
        if current_scroll == down_scroll:
            driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
            current_scroll = driver.execute_script('return window.pageYOffset')
            if current_scroll == down_scroll:
                error = True
                break
            else:
                end_crawling = True
                break
        try:
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()
            driver.switch_to.window(driver.window_handles[0])
            craw()
            driver.back()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[0])
            break
        except:
            return page_down()
    return

#사이트(메인) => 회사 채용고공 > 스크롤 내리기
def page_down2():
    global error
    global end_crawling
    while True:
        try:
            current_scroll2 = driver.execute_script('return window.pageYOffset')
            driver.execute_script(f'window.scrollTo(0, {current_scroll2 - 50});')
            time.sleep(1)
            driver.execute_script(f'window.scrollTo(0, {current_scroll2 + 150});')
            time.sleep(1)
            down_scroll2 = driver.execute_script('return window.pageYOffset')
            time.sleep(1)
            if current_scroll2 == down_scroll2:
                error = True
                break
            try:
                driver.find_element(By.XPATH, value='//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button').click()
                break
            except:
                return page_down2()
        except:
            return
    return


#데이터 크롤링 함수
def craw():
    for j in range(1,7):
        time.sleep(3)
        if j == 1:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')

                name.append(element.text.split('\n')[0].split('∙')[0])
                loc.append(element.text.split('\n')[0].split('∙')[1])
                career.append(element.text.split('\n')[0].split('∙')[2])
                title.append(element.text.split('\n')[1])
            except:
                name.append('')
                loc.append('')
                career.append('')
                title.append('')
        elif j == 2:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
                if len(element.text.split('\n')) >=1:
                    tag.append(element.text.split('\n')[1:])
                else:
                    tag.append('')
            except:
                tag.append('')
        elif j == 3:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')
                if len(element.text.split('\n')) ==2:
                    timeout.append(element.text.split('\n')[1])
                else:
                    timeout.append('')
            except:
                timeout.append('')
        elif j ==4:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')
                if len(element.text.split('\n')) == 2:
                    loc2.append(element.text.split('\n')[1])
                else:
                    loc2.append('')
            except:
                loc2.append('')
        elif j ==5:
            try:
                element = driver.find_element(By.CLASS_NAME, value='JobSkillTags_JobSkillTags__V1tZr')  
                if len(element.text.split('\n')) >=1:
                    skill.append(element.text.split('\n')[1:])
                else:
                    skill.append('')
            except:
                skill.append('')
        else:
            try:
                element = driver.find_element(By.CLASS_NAME, value='JobDescription_JobDescription__VWfcb')
                if '상세 정보 더 보기' in element.text:
                        page_down2()
                        element = driver.find_element(By.CLASS_NAME, value='JobDescription_JobDescription__VWfcb')
                        time.sleep(1)
                        detail.append(element.text)
            except:
                detail.append('')
                

#사이트 마다 변수 초기화
def reset():
    global title
    global name
    global career
    global loc
    global tag
    global timeout
    global loc2
    global detail
    global skill

    global done

    global current_scroll
    global down_scroll
    
    global current_scroll2
    global down_scroll2
    
    
    
    title = []           #채용공고
    name = []            #기업명
    career = []          #경력
    loc = []             #근무지
    tag = []             #태그
    timeout = []         #마감일
    loc2 = []            #근무지(상세)
    detail = []          #내용상세
    skill = []           #기술태그


    done = False

    # 스크롤 이동변수
    current_scroll = 0
    down_scroll = 0
    current_scroll2 = 0
    down_scroll2 = 0

#크롤링한 데이터 DF로 만들고 저장
def DF_make(file_name):
    tag_str = []
    skill_str = []
    df = pd.DataFrame()
    for i in tag:
        tag_str.append(','.join(i))
    for k in skill:
        skill_str.append(','.join(k))

    df['기업명'] = name
    df['채용공고'] = title
    df['스킬'] = skill_str
    df['위치'] = loc
    df['위치상세'] = loc2
    df['태그'] = tag_str
    df['마감일'] = timeout
    df['경력'] = career
    df['채용상세'] = detail
    
    
    df = df.drop_duplicates()
    df = df.fillna('')
    
    #파일 저장 위치 바꿔주세요 
    df.to_csv(f'C:\\Users\\EZEN\pro\\pro_data\\wanted_{file_name}.csv',encoding='utf-8')

#저장한 DF파일 불러와서 로컬DB에 저장 
#수정할것
#1.파일위치
#2.테이블명
def DB_insert(file_name):
    df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\wanted_{file_name}.csv',encoding='utf-8')
    df = df.fillna('')
    conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
    cur = conn.cursor()
    cur.execute(f'''CREATE TABLE IF NOT EXISTS wanted_{file_name} (
                   name VARCHAR(255),
                   title VARCHAR(255),
                   skill VARCHAR(255),
                   time VARCHAR(255),
                   loc VARCHAR(255),
                   career VARCHAR(255),
                   info VARCHAR(255),
                   loc2 VARCHAR(255),
                   detail TEXT(20000)
                )''')

    for index, row in df.iterrows():
        cur.execute(f"""
            INSERT INTO wanted_{file_name} (
               name,
               title,
               skill,
               time,
               loc,
               career,
               info,
               loc2,
               detail
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['기업명'],
            row['채용공고'],
            row['스킬'],
            row['마감일'],
            row['위치'],
            row['경력'],
            row['태그'],
            row['위치상세'],
            row['채용상세']
        ))
    # 변경사항 커밋
    conn.commit()

    # 연결 종료
    conn.close()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#크롤링 하고싶은 직무 선택후 url추가    
url_list = ['https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=seoul.gangnam-gu&locations=seoul.guro-gu&locations=seoul.geumcheon-gu&locations=seoul.seocho-gu&locations=seoul.songpa-gu',
            'https://www.wanted.co.kr/wdlist/518/873?country=kr&job_sort=job.recommend_order&years=-1&selected=873&locations=seoul.gangnam-gu&locations=seoul.guro-gu&locations=seoul.geumcheon-gu&locations=seoul.seocho-gu&locations=seoul.songpa-gu',
            'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&selected=655&locations=seoul.gangnam-gu&locations=seoul.guro-gu&locations=seoul.geumcheon-gu&locations=seoul.seocho-gu&locations=seoul.songpa-gu']

#저장될 파일명 지정
#기본: 원티드/머신러닝엔지니어URL/파일명ML_engineer
#리스트에 여러직무 추가가능
#url이 2개면 파일명도 2개
url_name_list = ['soft_engineer',
                 'web_dev',
                 'data_engineer']
file_url_name = dict(zip(url_list,url_name_list))   
    
    
    
#메인 실행 

end_num = 10 #크롤링할 데이터 개수 99999로 해도 문제없음(클롤링하다 데이터 없을경우 종료)

for page_url,file_name in file_url_name.items():
    driver = webdriver.Chrome("")
    time.sleep(10)
    driver.get(page_url)
    time.sleep(10)
    error = False
    page_num2 = 1
    end_crawling = False
    reset()
    while True:
        try:
            for page_num in tqdm(range(page_num2,end_num+1)):
                if(page_num % 20) == 0:
                    time.sleep(5)
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(5)
                    driver.quit()
                    time.sleep(5)
                    driver = webdriver.Chrome("")
                    time.sleep(5)
                    driver.get(page_url)
                    time.sleep(5)
                try:
                    driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()
                    driver.switch_to.window(driver.window_handles[0])
                    craw()
                    driver.back()
                    time.sleep(3)
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    page_down()
                if error == True:
                    print(에러만들기)
                elif end_crawling == True:
                    break

        except:
            time.sleep(5)
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(5)
            driver.quit()
            time.sleep(5)
            driver = webdriver.Chrome("")
            time.sleep(5)
            driver.get(page_url)
            time.sleep(5)
            page_num2 = page_num

        if (end_crawling ==True) or (page_num ==end_num):
            DF_make(file_name)
            DB_insert(file_name)
            page_num2 = 1
            break

십육
십칠
십팔
십구
십팔


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

a
일
1
2
3
4
5
2100
2200
2300
2500
2600
2100
2200
2300
2500
이


 10%|████████▎                                                                          | 1/10 [00:30<04:32, 30.25s/it]

a
일
1
2
3
4
5
2100
2200
2300
2500
2600
2100
2200
2300
2500
이


...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [04:32<00:30, 30.29s/it]

a
일
1
2
3
4
5
2100
2200
2300
2500
2600
2100
2200
2300
2500
이


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:03<00:00, 30.32s/it]


칠


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\EZEN\\pro\\pro_data\\pro_test.csv'